# Enunciado

**Utilizar un Codificador Transformer para clasificar las noticias de Reuters** en **46 temas mutuamente excluyentes**. Como cada noticia debe clasificarse en una sola categoría, es un problema de **"clasificación multiclase de una sola etiqueta"**.

El Reuters dataset es un conjunto de noticias breves y sus temas, publicado por Reuters en 1986. Son 46 temas diferentes; **algunos temas están más representados que otros**, pero cada uno tiene, al menos, **10 ejemplos en el conjunto** de entrenamiento.

Al igual que IMDB y MNIST, el conjunto de datos de Reuters viene **empaquetado como parte de Keras**.

# Desarrollo

Crearemos lo que se llama **Transformer Encoder**, uno de los componentes básicos de la arquitectura de Transformer, y lo aplicaremos a la tarea de clasificación de noticias de Reuters.

El siguiente código implementará un Transformer Encoder para clasificar noticias de Reuters en 46 categorías utilizando TensorFlow y Keras. Se prestará especial atención al entrenamiento del modelo y el ajuste de los hiperparámetros, según sea necesario.

Primero, necesitamos importar los módulos necesarios y definir algunas constantes:

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.datasets import reuters
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import TextVectorization, MultiHeadAttention

**Cargar y preparar los datos**

In [2]:
# Cargar el dataset Reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

# Dividir el conjunto de entrenamiento en subconjuntos de entrenamiento y validación
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=1337)

# Preparar los conjuntos de datos para su uso con Keras
maxlen = 500
train_data = keras.preprocessing.sequence.pad_sequences(train_data, maxlen=maxlen)
val_data = keras.preprocessing.sequence.pad_sequences(val_data, maxlen=maxlen)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, maxlen=maxlen)

# Convertir las etiquetas a formato one-hot
num_classes = np.max(train_labels) + 1
train_labels = keras.utils.to_categorical(train_labels, num_classes)
val_labels = keras.utils.to_categorical(val_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

# Crear datasets con los conjuntos de entrenamiento, validación y prueba
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels)).batch(batch_size)

2110848/2110848 [==============================] - 0s 0us/step


**Vectorizar los datos**

In [3]:
# Define the text vectorization parameters
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

# Obtener los datos y las etiquetas del conjunto de entrenamiento
train_texts = []
train_labels = []
for x, y in train_dataset:
    train_texts.extend([" ".join([str(word_index) for word_index in sample]) for sample in x.numpy()])
    train_labels.extend(y.numpy())

# Adapt the TextVectorization layer to the training texts
text_vectorization.adapt(train_texts)

# Crear dataset de entrenamiento con texto vectorizado
int_train_ds = text_vectorization(np.array(train_texts))

# Crear dataset de entrenamiento final
int_train_ds = tf.data.Dataset.from_tensor_slices((int_train_ds, train_labels)).batch(batch_size)

### El Transformer Encoder

En el ámbito del procesamiento de lenguaje natural, especialmente para tareas como la clasificación de textos —por ejemplo, determinar si una reseña de película es positiva o negativa utilizando datasets como reuters.load_data() en Keras—, tres componentes juegan roles fundamentales:

1. *Positional Embedding*: Esta técnica aporta información sobre el orden de las palabras dentro de una secuencia de texto a los modelos de transformadores, que por su diseño no capturan secuencialidad de forma inherente. Al sumar la información posicional al embedding de cada palabra, el modelo puede entender la estructura del texto y la relevancia del orden de las palabras, lo cual es crucial para interpretar correctamente el significado de las oraciones y párrafos.

2. *Transformer Encoder*: Constituye el bloque constructivo central de un modelo de transformador, procesando secuencias de entrada para crear representaciones contextualizadas del texto. Compuesto por múltiples capas que incluyen atención auto-dirigida y redes neuronales feed-forward, el encoder permite al modelo ponderar la importancia relativa de diferentes palabras basándose en su contexto. Esto facilita una comprensión profunda del texto, crucial para analizar y clasificar reseñas de películas donde el contexto de cómo se usan las palabras puede cambiar completamente su significado.

3. *Embedding*: Los embeddings transforman palabras en vectores de números, proporcionando una representación densa y significativa de cada palabra basada en su uso y contexto lingüístico. Estos vectores son esenciales para que los modelos de aprendizaje automático procesen texto, ya que les permiten operar con datos numéricos y aprender patrones asociados con diferentes clasificaciones de texto. Los embeddings pueden ser específicamente ajustados para una tarea, mejorando la capacidad del modelo para entender y clasificar correctamente las reseñas de películas según su contenido.

En conjunto, estos componentes habilitan a los modelos de transformadores para analizar textos complejos, como reseñas de películas, capturando tanto la estructura lingüística como el significado contextual de las palabras, lo cual es vital para realizar clasificaciones precisas y significativas.

**Transformer Encoder implementado como una subclase de `Layer`**

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # Tamaño de los vectores de los tokens de entrada
        self.embed_dim = embed_dim
        # Tamaño de la capa densa interna
        self.dense_dim = dense_dim
        # Número de attention heads
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="sigmoid"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    # El cálculo va en call()
    def call(self, inputs, mask=None):
        # La máscara que generará la capa Embedding
        # será 2D, pero la capa de atención espera
        # ser 3D o 4D, por lo que ampliamos su rango
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    # Implementamos la serialización para
    # que podamos guardar el modelo
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

VARIACIÓN

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
vocab_size = 10000
embed_dim = 256
num_heads = 2
dense_dim = 32
num_classes = 46  # Asumiendo que hay 46 clases únicas en el conjunto de datos de Reuters

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Configuración de callbacks para la regularización y optimización del entrenamiento
model_callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder_best.keras", save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),  # Aumento de la paciencia
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)  # Ajuste dinámico de la tasa de aprendizaje
]

model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=model_callbacks)

model = keras.models.load_model("transformer_encoder.keras", custom_objects={"TransformerEncoder": TransformerEncoder})
# Evaluar el modelo en el conjunto de datos de prueba
loss, accuracy = model.evaluate(test_dataset)

print(f"Loss en el conjunto de prueba: {loss}")
print(f"Precisión en el conjunto de prueba: {accuracy}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         2560000   
                                                                 
 transformer_encoder (Trans  (None, None, 256)         543776    
 formerEncoder)                                                  
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 46)                11822 

KeyboardInterrupt: 

Ahora, implementaremos el Transformer Encoder y la incrustación posicional como subclases de Layer:

In [6]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**Usando el Transformer encoder para clasificación de texto**

Finalmente, para crear y entrenar un modelo combinando el Transformer Encoder con incrustación posicional (positional embedding) con el fin de conseguir un modelo de clasifiación de texto, primero necesitamos definir el modelo en Keras. Luego, podemos compilarlo y entrenarlo utilizando los datos preparados. Aquí está el proceso paso a paso:

Definir el modelo:

Añadir una capa de entrada para los datos de entrada.
Añadir una capa de incrustación posicional para inyectar información de posición.
Añadir un TransformerEncoder para procesar las secuencias de entrada.
Añadir una capa de reducción de dimensionalidad como GlobalMaxPooling1D o similar.
Añadir una capa Dropout para regularización.
Añadir una capa de salida con activación softmax para la clasificación multiclase.
Compilar el modelo:

Especificar la función de pérdida (por ejemplo, categorical_crossentropy para la clasificación multiclase).
Seleccionar un optimizador (por ejemplo, rmsprop).
Especificar las métricas a seguir durante el entrenamiento (por ejemplo, accuracy).
Entrenar el modelo:

Utilizar los conjuntos de datos preparados (train_dataset, val_dataset, test_dataset) para entrenamiento, validación y evaluación respectivamente.
Especificar el número de épocas y cualquier otro hiperparámetro relevante.
Aquí está el código para llevar a cabo estos pasos:

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)  # Cambiado a num_classes

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",  # Cambiado a categorical_crossentropy
              metrics=["accuracy"])
model.summary()

**Entrenamiento y evaluación del modelo basado en Transformer Encoder**

In [ ]:
# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

Antes de continuar el documento y pasar a probar nuevas modificaciones de los modelos, extraeremos ciertas conclusiones de estos dos primeros.

El primer modelo, en el que se trabaja con *Embedding*, se ronda el 81% de precisión. Por tanto, debido a que el código base estudiado en clase ronda el 70%, es posible afirmar que la elección en la variación de parámetros ha conllevado un aumento en la nueva precisión.

## Ejemplos alternativos

### Variación de hiperparámetros

In [ ]:
vocab_size = 10000
sequence_length = 600
embed_dim = 256
num_heads = 4 # Ejemplo 1: Cambiar el número de cabezas de atención en el TransformerEncoder
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posi  (None, None, 256)         2713600   
 tionalEmbedding)                                                
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         1069600   
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

In [ ]:
vocab_size = 10000
sequence_length = 600
embed_dim = 512 # Ejemplo 2: Modificar la dimensionalidad de las incrustaciones
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

### Arquitecturas alternativas

In [ ]:
vocab_size = 10000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
# Ejemplo 1: Añadir capas convolucionales después del TransformerEncoder
x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
# Ejemplo 2: Utilizar una red neuronal recurrente (LSTM) en lugar del TransformerEncoder
x = layers.LSTM(128)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
# Ejemplo 3: Combinar una CNN con una capa LSTM
cnn_output = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
lstm_output = layers.LSTM(64)(cnn_output)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

### Técnicas de regularización

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
# Ejemplo 1: Añadir regularización L2 a las capas densas
from keras import regularizers
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

### Cambiar el optimizador

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="adam", # Ejemplo 1
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

### Ajustar la tasa de aprendizaje

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

### Añadir programación de tasa de aprendizaje

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer,
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

### Aumentar el número de épocas

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=30, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

### Utilizar callbacks adicionales

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# Entrenar el modelo
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_with_positional_embedding.keras",
                                    save_best_only=True),
    keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2)
]
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=callbacks)

# Evaluar el modelo
model = keras.models.load_model(
    "transformer_with_positional_embedding.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test accuracy: {model.evaluate(test_dataset)[1]:.3f}")

Hemos concluido que  la proliferación de modelos nos ayuda a tener una visión amplia de los reusltado. Sin embargo, debido a la gran carga computacional y al entorno no es capaz de soportarlo por ello hemos decidido no realizarlo en este ejercicio. En un entorno mejor donde se pudiera ver con claridad, se podría observar las diferencias entre modelos por ello hemos querido dejar todos los modelos creados para que se pueda ver el trabajo detrás de ellos, aunque no se hayan podido compilar

No obstante, se pueden sacar conclusiones determinantes con el fin de encontrar el mejor modelo a partir de la estratégica modificación de las especificaciones de los modelos y el cambio en la precisión final que estos provocan